In [3]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [4]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config

In [13]:
def print_sent_ie(sp_i, d):
    print("sentence{}".format(sp_i))
    print(d['SENTS'][sp_i]['text'])
    for e in d['SENTS'][sp_i]['IE']['NER']:
        print(e)

In [8]:
data = json_load("FGC_release_all_test.json")

In [37]:
data[0]

{'DID': 'D003',
 'QUESTIONS': [{'QID': 'D003Q02',
   'QTYPE': '基礎題',
   'ATYPE': 'Event',
   'AMODE': ['Single-Span-Extraction'],
   'QTEXT': '蘇東坡因為什麼事件入獄幾乎要死在獄中?',
   'QTEXT_CN': '苏东坡因为什么事件入狱几乎要死在狱中?',
   'SENTS': [{'text': '苏东坡因为什么事件入狱几乎要死在狱中?',
     'start': 0,
     'end': 19,
     'IE': {'NER': [{'id': 'D0-S0-M0',
        'string': '苏东坡',
        'type': 'PER',
        'char_b': 0,
        'char_e': 3}],
      'COREF': {},
      'RELATION': [],
      'TOKEN': [{'word': '苏东坡', 'char_b': 0, 'char_e': 3, 'pos': 'NR'},
       {'word': '因为', 'char_b': 3, 'char_e': 5, 'pos': 'P'},
       {'word': '什么', 'char_b': 5, 'char_e': 7, 'pos': 'DT'},
       {'word': '事件', 'char_b': 7, 'char_e': 9, 'pos': 'NN'},
       {'word': '入狱', 'char_b': 9, 'char_e': 11, 'pos': 'VV'},
       {'word': '几乎', 'char_b': 11, 'char_e': 13, 'pos': 'AD'},
       {'word': '要死', 'char_b': 13, 'char_e': 15, 'pos': 'VV'},
       {'word': '在', 'char_b': 15, 'char_e': 16, 'pos': 'P'},
       {'word': '狱中', 'char_b': 16, '

In [36]:
len(data)

193

In [106]:
single_num = 0
other_q = 0

answer_in = 0
answer_out = 0 
for d in data:
    for q in d['QUESTIONS']:
        if q['QTYPE'] == '申論':
            other_q += 1
        
        elif 'Single-Span-Extraction' in q['AMODE']:
            single_num += 1
            sp_text = ""
            for s_i, s in enumerate(d['SENTS']):
                if s_i in q['sp']:
                    sp_text += s['text']
            shint_text = ""
            for s_i , s in enumerate(d['SENTS']):
                if s_i in q['SHINT']:
                    shint_text += s['text']
            for answer in q['ANSWER']:
                if answer['ATEXT_CN'] in shint_text:
                    if answer['ATEXT_CN'] not in sp_text:
                        answer_out += 1
                        print(q['QID'])
                        break
        else:
            other_q += 1

D003Q05
D003Q07
D033Q04
D033Q06
D039Q08
D069Q08
D087Q01
D087Q06
D093Q06
D093Q07
D105Q10
D117Q01
D117Q06
D183Q02
D243Q01
D243Q04
D249Q03
D255Q08
D255Q10
D261Q05
D267Q04
D291Q01
D291Q08
D303Q01
D303Q04
D315Q01
D315Q03
D315Q07
D315Q08


In [107]:
yesno_q_num

25

In [110]:
single_num

134

In [111]:
answer_out

29

In [70]:
def find_q(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q)

In [114]:
find_q('D093Q07', data)

{'QID': 'D093Q07', 'QTYPE': '基礎題', 'ATYPE': 'Object', 'AMODE': ['Single-Span-Extraction'], 'QTEXT': '台灣地區使用的中文是何種字體?', 'QTEXT_CN': '台湾地区使用的中文是何种字体?', 'SENTS': [{'text': '台湾地区使用的中文是何种字体?', 'start': 0, 'end': 15, 'IE': {'NER': [{'id': 'D0-S0-M0', 'string': '台湾', 'type': 'STATE_OR_PROVINCE', 'char_b': 0, 'char_e': 2}, {'id': 'D0-S0-M1', 'string': '中文', 'type': 'DEMONYM', 'char_b': 7, 'char_e': 9}], 'COREF': {}, 'RELATION': [], 'TOKEN': [{'word': '台湾', 'char_b': 0, 'char_e': 2, 'pos': 'NR'}, {'word': '地区', 'char_b': 2, 'char_e': 4, 'pos': 'NN'}, {'word': '使用', 'char_b': 4, 'char_e': 6, 'pos': 'VV'}, {'word': '的', 'char_b': 6, 'char_e': 7, 'pos': 'DEC'}, {'word': '中文', 'char_b': 7, 'char_e': 9, 'pos': 'NN'}, {'word': '是', 'char_b': 9, 'char_e': 10, 'pos': 'VC'}, {'word': '何', 'char_b': 10, 'char_e': 11, 'pos': 'DT'}, {'word': '种', 'char_b': 11, 'char_e': 12, 'pos': 'M'}, {'word': '字体', 'char_b': 12, 'char_e': 14, 'pos': 'NN'}, {'word': '?', 'char_b': 14, 'char_e': 15, 'pos': 'PU'}]}}], 'SHI

In [57]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("sp:{}".format(q['sp']))
                print("answer:{}".format(q['ANSWER']))
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print_sent_ie(sp_i, d)
                    print()

In [132]:
print_analysis_from_qid('D105Q01', data)

D105Q01
全民健保于哪一年实施?
atype:Date-Duration
SHINT:[0, 1, 35, 36]
sp:[16, 24, 33, 36]
answer:[{'ATEXT': '1995年', 'ATEXT_CN': '1995年', 'ATOKEN': [{'text': '1995年', 'text_cn': '1995年', 'start': 510, 'end': 515}]}]
sentence0
全民健康保险，

sentence1
一般简称为「全民健保」或「健保」，

sentence2
是一种强制性保险的福利政策，
{'id': 'D0-S0-M0', 'string': '一', 'type': 'NUMBER', 'char_b': 1, 'char_e': 2}

sentence3
法源是依据《中华民国宪法增修条文》所实施的全民医疗保险制度。
{'id': 'D0-S0-M0', 'string': '中华民国宪法增修条文', 'type': 'MISC', 'char_b': 6, 'char_e': 16}

sentence4

在全民健康保险实施以前，

sentence5
台湾按照职业别分别实施军人保险（军保）、公教人员保险（公保）、劳工保险（劳保）、农民健康保险（农保），
{'id': 'D0-S0-M0', 'string': '台湾', 'type': 'STATE_OR_PROVINCE', 'char_b': 0, 'char_e': 2}
{'id': 'D0-S0-M1', 'string': '公保', 'type': 'PER', 'char_b': 27, 'char_e': 29}
{'id': 'D0-S0-M2', 'string': '农民', 'type': 'TITLE', 'char_b': 40, 'char_e': 42}

sentence6
并有保险业各自承保的健康保险。

sentence7
 1986年5月，
{'id': 'D0-S0-M0', 'string': '1986年5月', 'type': 'DATE', 'char_b': 1, 'char_e': 8}

sentence8
行政院核定的「中华民国台湾经济长期展望」中提到以2000年为全民健保开办目

In [33]:
d = data[2]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("sp:{}".format(q['sp']))
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print_sent_ie(sp_i, d)
        print()

D003Q03
在「乌台诗案」中，有哪些人出面救了苏东坡？
atype:Person
SHINT:[19, 20, 21]
sp:[15, 20]
sentence15
苏轼参加救灾。

sentence16

元丰二年（1079年），
{'id': 'D0-S0-M0', 'string': '元丰', 'type': 'MISC', 'char_b': 1, 'char_e': 3}
{'id': 'D0-S0-M1', 'string': '二年', 'type': 'DATE', 'char_b': 3, 'char_e': 5}
{'id': 'D0-S0-M2', 'string': '1079年', 'type': 'DATE', 'char_b': 6, 'char_e': 11}

sentence17
四十三岁时，
{'id': 'D0-S0-M0', 'string': '四十三', 'type': 'NUMBER', 'char_b': 0, 'char_e': 3}
{'id': 'D0-S0-M1', 'string': '岁', 'type': 'MISC', 'char_b': 3, 'char_e': 4}

sentence18
因乌台诗案入狱，

sentence19
几死，因为写文章向朝廷诀别，
{'id': 'D0-S0-M0', 'string': '几', 'type': 'NUMBER', 'char_b': 0, 'char_e': 1}

sentence20
太皇太后曹氏、王安礼等人出面力挽，
{'id': 'D0-S0-M0', 'string': '王安礼', 'type': 'PER', 'char_b': 7, 'char_e': 10}

sentence21
神宗动心，苏轼终免一死，
{'id': 'D0-S0-M0', 'string': '苏轼终', 'type': 'PER', 'char_b': 5, 'char_e': 8}
{'id': 'D0-S0-M1', 'string': '一', 'type': 'NUMBER', 'char_b': 9, 'char_e': 10}



In [26]:
for d_i, d in enumerate(data):
    for q in d['QUESTIONS']:
        print(d_i)
        print(q['QTEXT_CN'])

0
上官均是否赞成王安石的新政改革?
1
苏东坡因为什么事件入狱几乎要死在狱中?
2
在「乌台诗案」中，有哪些人出面救了苏东坡？
3
苏东坡因「乌台诗案」被贬到什么地方？
4
宋神宗熙宁十月，黄河溃堤时，苏东坡人在哪里？
5
苏东坡被贬至黄州时，认识了哪些朋友？
6
西元1079年时徐州发生水灾的原因为何？
7
华尔街位于美国哪一个城市?
8
「占领华尔街」运动是由哪一个组织所发起?
9
「占领华尔街」运动的诉求为何?
10
「占领华尔街」运动延烧至美国哪些城市?
11
请列举出三项高雄的地标建物?
12
高雄市被列为国定古迹的建筑物是哪些?
13
高雄流行文化中心在哪里?
14
高雄有哪些大型购物中心?
15
高雄的大型购物中心有多少个?
16
高雄总共有多少座古迹?
17
「北美自由贸易协议2.0」的英文全名为何?
18
「北美自由贸易协议2.0」的英文简称为何?
19
「北美自由贸易协议2.0」的法文全名为何?
20
「北美自由贸易协议2.0」的法文简称为何?
21
「北美自由贸易协议2.0」的西班牙文全名为何?
22
「北美自由贸易协议2.0」的西班牙文简称为何?
23
「北美自由贸易协议2.0」的中文全名为何?
24
「北美自由贸易协议2.0」的中文简称为何?
25

9. 「北美自由贸易协议2.0」是哪三个国家之间的协议?
26
「北美自由贸易协议2.0」由哪三个国家的元首共同签署?
27
「北美自由贸易协议2.0」在何时签署?
28
「北美自由贸易协议2.0」在何地签署?
29
代表美国签署「北美自由贸易协议2.0」的人是谁?
30
代表墨西哥签署「北美自由贸易协议2.0」的人是谁?
31
代表加拿大签署「北美自由贸易协议2.0」的人是谁?
32
结婚男女对自己男性伴侣的称呼有哪些?
33
结婚男女对自己女性伴侣的称呼有哪些?
34
结婚后要如何称呼先生的父亲?
35
结婚后要如何称呼先生的母亲?
36
结婚后要如何称呼太太的父亲?
37
结婚后要如何称呼儿子的太太?(亲属关系称谓)
38
结婚后要如何称呼女儿的丈夫?(亲属关系称谓)
39
岳父指的是谁的父亲?
40
岳母指的是谁的母亲?
41
公公指的是谁的父亲?
42
婆婆指的是谁的母亲?
43
QQ平台是中国哪一家公司的？
44
QQ平台2017年推出那两款聊天机器人?
45
QQ小冰是哪一家公